In [242]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf

In [243]:
tsla = yf.Ticker("tsla")

tsla_hist = tsla.history(period='5y', interval='1d', end='2025-06-13')
# tsla_hist

In [244]:
tsla_hist.drop(columns=['Dividends', 'Stock Splits'], inplace=True)

In [245]:
tsla_hist.index =pd.to_numeric(tsla_hist.index)
tsla_hist.index = tsla_hist.index/(max(tsla_hist.index)) + 1

In [246]:
for col in tsla_hist.columns:
    tsla_hist[col] = tsla_hist[col]/max(tsla_hist[col]) + 1

In [247]:
X = tsla_hist.drop(columns=['Close'])
y = tsla_hist['Close']

In [248]:
X['Date'] = X.index

In [249]:
for delay in range(1,4):
    X[f"Delay {delay}"] = y.iloc[(3-delay):-(delay)]

In [250]:
X_sample = [[d, h, l, o, vol, v1, v2, v3] for d, h, l, o, vol, v1, v2, v3 in zip(X['Date'].iloc[3:].values, X['High'].iloc[3:].values, X['Low'].iloc[3:].values,X['Open'].iloc[3:].values, X['Volume'].iloc[3:].values, X['Delay 1'].dropna().values, X['Delay 2'].dropna().values, X['Delay 3'].dropna().values)]
y_sample = y.iloc[3:].values

In [251]:
len(X['Date'].iloc[3:].values), len(X_sample)

(1252, 1252)

In [252]:
len(y_sample), len(X_sample)

(1252, 1252)

In [253]:
torch.set_default_dtype(torch.float64)

In [254]:
X = torch.from_numpy(np.array(X))
y = torch.from_numpy(np.array(y))

In [255]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [256]:
train_data = [(X,y) for X, y in zip(X[:-273], y[:-273])]
test_data = [(X,y) for X, y in zip(X[-273:], y[-273:])]

In [ ]:
temp_tensor = train_data[0][0]
temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
temp_tuple_1 = (temp_tensor, train_data[0][1])


temp_tensor = train_data[1][0]
temp_tensor = torch.nan_to_num(temp_tensor, nan=1)
temp_tuple_2 = (temp_tensor, train_data[1][1])

In [258]:
train_data[0] = temp_tuple_1
train_data[1] = temp_tuple_2

In [259]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=32,)
test_dataloader= DataLoader(test_data, batch_size=32,)

In [ ]:
count = 0
batch = 0
for (X, y) in train_dataloader:
    if (torch.any(X.isnan())):
        print(f"Found on batch: {batch}")
        print(X)
        print(torch.nan_to_num(X, nan=0))
        count+=1
    batch +=1


In [261]:
class MLP(nn.Module):
    def __init__(self, input_features, hidden_features, output_features):
        super().__init__()
        self.l1 = nn.Sequential(
            nn.Linear(input_features, hidden_features),
            nn.ReLU()
        )
        self.l2 = nn.Sequential(
            nn.Linear(hidden_features, 15),
            nn.ReLU()
        )
        self.l3 = nn.Linear(15, output_features)

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        return x

In [262]:
mlpmodel = MLP(8, 30, 1)

In [263]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.SGD(params=mlpmodel.parameters(), lr=0.01)

In [264]:
EPOCHS = 1

mlpmodel.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch, (X, y) in enumerate(train_dataloader):
        preds = mlpmodel(X)
        # print(f"Preds + {preds}")
        loss = loss_fn(torch.unsqueeze(y, 1), preds)
        total_loss+= loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"We're on batch: {batch} + Loss: {total_loss/batch}")


We're on batch: 0 + Loss: inf
We're on batch: 1 + Loss: 2.346954028012278
We're on batch: 2 + Loss: 1.7312961481401326
We're on batch: 3 + Loss: 1.544543119343059
We're on batch: 4 + Loss: 1.5111052117718473
We're on batch: 5 + Loss: 1.4236348373696486
We're on batch: 6 + Loss: 1.3365220997874905
We're on batch: 7 + Loss: 1.2428561409488184
We're on batch: 8 + Loss: 1.1672513913403293
We're on batch: 9 + Loss: 1.1012971995157639
We're on batch: 10 + Loss: 1.0569009193965964
We're on batch: 11 + Loss: 1.0138467609579536
We're on batch: 12 + Loss: 0.9620950308833703
We're on batch: 13 + Loss: 0.9059085610695831
We're on batch: 14 + Loss: 0.8543102186490612
We're on batch: 15 + Loss: 0.8012812215176187
We're on batch: 16 + Loss: 0.7551363760312493
We're on batch: 17 + Loss: 0.7135842634056954
We're on batch: 18 + Loss: 0.6743714729486949
We're on batch: 19 + Loss: 0.638929529806318
We're on batch: 20 + Loss: 0.6070866043051691
We're on batch: 21 + Loss: 0.5782633613483256
We're on batch: 

In [ ]:
with torch.inference_mode():
    mlpmodel.eval()
    
